Please run below cell to install all libraries required to run the codes below

In [ ]:
!pip install spacy
!pip install newspaper3k
!pip install wordcloud
!pip install textblob
!pip install seaborn
!python -m spacy download en_core_web_sm

In [69]:
import spacy
import pandas as pd
import json
import nltk
import en_core_web_sm
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize.regexp import WhitespaceTokenizer
import newspaper
from newspaper import article
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import base64
import string
import re
from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from spacy.lang.en import English
parser = English()
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aydin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aydin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv (r'test_agor.csv')
new_df = df[['Title', 'Body']] 
new_df.head()

,Title,Body
0,I will never kill myself. I have no intent or ...,[deleted]
1,I've been struggling with loneliness lately an...,[removed]
2,I'm happier on days when I cry and don't know ...,What the fuck
3,I found a job. (I'm not happy about it),Hi... so I got a job today. My depression has ...
4,They say talk to someone about it but no one r...,On websites and such they always recommend tha...


In [ ]:
#docs = list(nlp.pipe(doc))

In [5]:
# new_df = new_df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
# new_df.head()

,Title,Body
0,I will never kill myself. I have no intent or ...,[deleted]
1,I've been struggling with loneliness lately an...,[removed]
2,I'm happier on days when I cry and don't know ...,What the fuck
3,I found a job. (I'm not happy about it),Hi... so I got a job today. My depression has ...
4,They say talk to someone about it but no one r...,On websites and such they always recommend tha...


## Create list of useless words that is made of stopwords and punctuations 

In [140]:
punctuations = string.punctuation
useless_words = nltk.corpus.stopwords.words("english") + list(punctuations)

## Extend list of useless words with additional words that are found in our list of words

In [224]:
useless_words_extended = useless_words.copy()
useless_words_extended.extend(["n't","'m","'s", "'ve", "'ll", "'re", "...", "..", '``', "''"])

In [225]:
# Quick sanity check to see whether the list of useless words contain all the words we want
useless_words_extended

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [226]:
a = new_df['Title'].to_list()
b = new_df['Body'].to_list()
corpus = [''.join(map(str, i)) for i in zip(a, b)]

# loop through each element in the list (in this case the list made of string elements) and tokenize each line and add it to the list of words
list_of_words = []
for line in corpus:
    list_of_words.extend(nltk.word_tokenize(line.lower()))
list_of_words

['i',
 'will',
 'never',
 'kill',
 'myself',
 '.',
 'i',
 'have',
 'no',
 'intent',
 'or',
 'desire',
 '.',
 'all',
 'i',
 "'ve",
 'ever',
 'wanted',
 'to',
 'do',
 'is',
 'move',
 'on',
 'with',
 'my',
 'fucking',
 'life',
 '!',
 '[',
 'deleted',
 ']',
 'i',
 "'ve",
 'been',
 'struggling',
 'with',
 'loneliness',
 'lately',
 'and',
 'feeling',
 'like',
 'i',
 "'m",
 'lesser',
 'than',
 'others',
 '.',
 'this',
 'text',
 'helped',
 'me',
 'tremendously',
 '.',
 '[',
 'removed',
 ']',
 'i',
 "'m",
 'happier',
 'on',
 'days',
 'when',
 'i',
 'cry',
 'and',
 'do',
 "n't",
 'know',
 'why.what',
 'the',
 'fuck',
 'i',
 'found',
 'a',
 'job',
 '.',
 '(',
 'i',
 "'m",
 'not',
 'happy',
 'about',
 'it',
 ')',
 'hi',
 '...',
 'so',
 'i',
 'got',
 'a',
 'job',
 'today',
 '.',
 'my',
 'depression',
 'has',
 'kicked',
 'in',
 'very',
 'bad',
 'now',
 'about',
 'it',
 '.',
 'see',
 ',',
 'i',
 'do',
 "n't",
 'really',
 'want',
 'to',
 'work',
 '.',
 'it',
 "'s",
 'not',
 'because',
 'i',
 "'m",
 'l

In [227]:
filtered_words = [word for word in list_of_words if not word in useless_words_extended]
filtered_words

['never',
 'kill',
 'intent',
 'desire',
 'ever',
 'wanted',
 'move',
 'fucking',
 'life',
 'deleted',
 'struggling',
 'loneliness',
 'lately',
 'feeling',
 'like',
 'lesser',
 'others',
 'text',
 'helped',
 'tremendously',
 'removed',
 'happier',
 'days',
 'cry',
 'know',
 'why.what',
 'fuck',
 'found',
 'job',
 'happy',
 'hi',
 'got',
 'job',
 'today',
 'depression',
 'kicked',
 'bad',
 'see',
 'really',
 'want',
 'work',
 'lazy',
 'dislike',
 'idea',
 'working',
 'see',
 'point',
 'pretty',
 'mentally',
 'fucked',
 'ptsd',
 'dissociative',
 'disorder',
 'depression',
 'anxiety',
 'ect',
 'ect',
 'makes',
 'nearly',
 'anything',
 'massive',
 'difficulty',
 'top',
 'working',
 'minimum',
 'wage',
 'trying',
 'get',
 'abusive',
 'house',
 'feel',
 'like',
 'life',
 'going',
 'completely',
 'move',
 'full',
 'time',
 'hell',
 'feel',
 'like',
 'part',
 'time..',
 'free',
 'time',
 'save',
 'money',
 'worry',
 'work',
 'sleep',
 'schedules',
 'ca',
 'keep',
 'mental',
 'health',
 'loves'

In [228]:
word_counter = Counter(filtered_words)
word_counter

Counter({'never': 232,
         'kill': 58,
         'intent': 3,
         'desire': 12,
         'ever': 114,
         'wanted': 85,
         'move': 34,
         'fucking': 90,
         'life': 427,
         'deleted': 202,
         'struggling': 18,
         'loneliness': 12,
         'lately': 17,
         'feeling': 134,
         'like': 654,
         'lesser': 1,
         'others': 52,
         'text': 14,
         'helped': 24,
         'tremendously': 2,
         'removed': 53,
         'happier': 4,
         'days': 92,
         'cry': 40,
         'know': 404,
         'why.what': 1,
         'fuck': 50,
         'found': 43,
         'job': 96,
         'happy': 86,
         'hi': 4,
         'got': 167,
         'today': 64,
         'depression': 251,
         'kicked': 7,
         'bad': 85,
         'see': 111,
         'really': 267,
         'want': 374,
         'work': 107,
         'lazy': 16,
         'dislike': 1,
         'idea': 23,
         'working': 56,
     

In [229]:
most_common_words = word_counter.most_common()[:10]
most_common_words

[('like', 654),
 ('feel', 551),
 ('life', 427),
 ('know', 404),
 ('want', 374),
 ('time', 360),
 ('get', 315),
 ('even', 296),
 ('people', 278),
 ('would', 269)]

In [230]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aydin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [232]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

lemmatized_wordslist = []
for word in filtered_words:
    lem_str = lemmatizer.lemmatize(word)
    lemmatized_wordslist.append(lem_str)

lemmatized_wordslist

['never',
 'kill',
 'intent',
 'desire',
 'ever',
 'wanted',
 'move',
 'fucking',
 'life',
 'deleted',
 'struggling',
 'loneliness',
 'lately',
 'feeling',
 'like',
 'lesser',
 'others',
 'text',
 'helped',
 'tremendously',
 'removed',
 'happier',
 'day',
 'cry',
 'know',
 'why.what',
 'fuck',
 'found',
 'job',
 'happy',
 'hi',
 'got',
 'job',
 'today',
 'depression',
 'kicked',
 'bad',
 'see',
 'really',
 'want',
 'work',
 'lazy',
 'dislike',
 'idea',
 'working',
 'see',
 'point',
 'pretty',
 'mentally',
 'fucked',
 'ptsd',
 'dissociative',
 'disorder',
 'depression',
 'anxiety',
 'ect',
 'ect',
 'make',
 'nearly',
 'anything',
 'massive',
 'difficulty',
 'top',
 'working',
 'minimum',
 'wage',
 'trying',
 'get',
 'abusive',
 'house',
 'feel',
 'like',
 'life',
 'going',
 'completely',
 'move',
 'full',
 'time',
 'hell',
 'feel',
 'like',
 'part',
 'time..',
 'free',
 'time',
 'save',
 'money',
 'worry',
 'work',
 'sleep',
 'schedule',
 'ca',
 'keep',
 'mental',
 'health',
 'love',
 '

In [233]:
nlp = en_core_web_sm.load()

In [234]:
cleaned_wordlist = nlp(str(lemmatized_wordslist))

In [235]:
#def tokens_without_punct(clean_doc):
    #doc = [t.text for t in doc if t.pos_ != 'PUNCT']
    #return doc

In [236]:
#nlp.add_pipe(tokens_without_punct,after= 'tagger')

In [237]:
# lemmatized_doc = " ".join([token.lemma_ for token in str(filtered_words)])

In [238]:
token_list = [[token,token.pos_] for token in cleaned_wordlist]
tk = pd.DataFrame(token_list,columns = ['word','POS'])
tk = tk[tk.POS != 'PUNCT']
tk_verb = tk[tk.POS == 'VERB']


In [239]:
token_list = [[token,token.pos_] for token in cleaned_wordlist]
tk = pd.DataFrame(token_list,columns = ['word','POS'])
tk = tk[tk.POS != 'PUNCT']
tk_adj = tk[tk.POS == 'ADJ']

In [240]:
tk = tk[tk.POS != 'PUNCT']
tk_adj = tk[tk.POS == 'ADJ']
l1 = [g1.text for g1 in tk_adj.word.tolist()]
d =Counter(l1)
agora_adj_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_adj_df.columns = ['word','count']

agora_adj_df['POS'] =pd.Series(['ADJ' for i in range(agora_adj_df.shape[0])])
agora_adj_df = agora_adj_df.sort_values(by = 'count', ascending = False)
agora_adj_df.iloc[0:45]

,word,count,POS
23,much,203,ADJ
18,depressed,126,ADJ
73,good,126,ADJ
20,better,114,ADJ
114,last,100,ADJ
95,first,99,ADJ
51,hard,94,ADJ
138,long,87,ADJ
2,happy,86,ADJ
3,bad,86,ADJ


In [241]:
tk = tk[tk.POS != 'PUNCT']
tk_pnoun = tk[tk.POS == 'PROPN']
l1 = [g1.text for g1 in tk_pnoun.word.tolist()]
d =Counter(l1)
agora_pnoun_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_pnoun_df.columns = ['word','count']

agora_pnoun_df['POS'] =pd.Series(['PROPN' for i in range(agora_pnoun_df.shape[0])])
agora_pnoun_df = agora_pnoun_df.sort_values(by = 'count', ascending = False)
agora_pnoun_df.iloc[0:45]

,word,count,POS
539,_,209,PROPN
5,wish,68,PROPN
99,med,34,PROPN
96,miss,27,PROPN
15,young,23,PROPN
53,wan,21,PROPN
220,hang,20,PROPN
98,mg,17,PROPN
135,stuck,17,PROPN
54,god,16,PROPN


In [242]:
tk = tk[tk.POS != 'PUNCT']
tk_noun = tk[tk.POS == 'NOUN']
l1 = [g1.text for g1 in tk_noun.word.tolist()]
d =Counter(l1)
agora_noun_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_noun_df.columns = ['word','count']

agora_noun_df['POS'] =pd.Series(['NOUN' for i in range(agora_noun_df.shape[0])])
agora_noun_df = agora_noun_df.sort_values(by = 'count', ascending = False)
agora_noun_df.iloc[0:45]

,word,count,POS
5,life,437,NOUN
25,time,433,NOUN
80,year,332,NOUN
67,friend,326,NOUN
9,day,308,NOUN
58,people,280,NOUN
76,thing,273,NOUN
13,depression,262,NOUN
32,ca,221,NOUN
166,help,165,NOUN


In [243]:
l1 = [g1.text for g1 in tk_verb.word.tolist()]
d =Counter(l1)
agora_verb_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
agora_verb_df.columns = ['word','count']
agora_verb_df.sort_values(by='count', ascending=False)
agora_verb_df['POS'] =pd.Series(['VERB' for i in range(agora_verb_df.shape[0])])
agora_verb_df = agora_verb_df.sort_values(by = 'count', ascending = False)
agora_verb_df.iloc[:45
                  
                  ]

,word,count,POS
16,feel,626,VERB
6,know,420,VERB
10,want,349,VERB
55,would,272,VERB
66,think,233,VERB
14,make,231,VERB
41,go,210,VERB
1,deleted,202,VERB
17,going,198,VERB
3,feeling,178,VERB


In [244]:
#from collections import Counter
#Counter(token_list)

In [245]:
blob = TextBlob(clean_doc)

In [246]:
blob3 = blob.ngrams(n=3)
blob3

[WordList(['I', 'will', 'never']),
 WordList(['will', 'never', 'kill']),
 WordList(['never', 'kill', 'myself']),
 WordList(['kill', 'myself', 'I']),
 WordList(['myself', 'I', 'have']),
 WordList(['I', 'have', 'no']),
 WordList(['have', 'no', 'intent']),
 WordList(['no', 'intent', 'or']),
 WordList(['intent', 'or', 'desire']),
 WordList(['or', 'desire', 'All']),
 WordList(['desire', 'All', 'I']),
 WordList(['All', 'I', 've']),
 WordList(['I', 've', 'ever']),
 WordList(['ve', 'ever', 'wanted']),
 WordList(['ever', 'wanted', 'to']),
 WordList(['wanted', 'to', 'do']),
 WordList(['to', 'do', 'is']),
 WordList(['do', 'is', 'move']),
 WordList(['is', 'move', 'on']),
 WordList(['move', 'on', 'with']),
 WordList(['on', 'with', 'my']),
 WordList(['with', 'my', 'fucking']),
 WordList(['my', 'fucking', 'life']),
 WordList(['fucking', 'life', 'deleted']),
 WordList(['life', 'deleted', 'I']),
 WordList(['deleted', 'I', 've']),
 WordList(['I', 've', 'been']),
 WordList(['ve', 'been', 'struggling']),


In [247]:
flat_blob3 = [bb[0]+' '+bb[1]+' ' +bb[2] for bb in blob3]
Counter(flat_blob3).most_common(100)

[('I don t', 367),
 ('don t know', 143),
 ('I can t', 132),
 ('I feel like', 108),
 ('I want to', 94),
 ('t want to', 93),
 ('don t want', 91),
 ('I ve been', 74),
 ('I m not', 69),
 ('feel like I', 65),
 ('I have no', 62),
 ('and I m', 58),
 ('t know what', 58),
 ('don t have', 57),
 ('I didn t', 51),
 ('a lot of', 48),
 ('just want to', 47),
 ('what to do', 46),
 ('in my life', 45),
 ('I m just', 45),
 ('I just want', 44),
 ('I ’ m', 44),
 ('want to be', 43),
 ('that I m', 42),
 ('know what to', 41),
 ('and I don', 36),
 ('to talk to', 35),
 ('I have a', 34),
 ('of my life', 34),
 ('I have to', 33),
 ('just don t', 33),
 ('I m a', 32),
 ('when I was', 31),
 ('I haven t', 31),
 ('like I m', 30),
 ('all the time', 30),
 ('my life I', 29),
 ('be able to', 29),
 ('but I m', 29),
 ('don ’ t', 29),
 ('don t feel', 28),
 ('know how to', 28),
 ('that I have', 28),
 ('I know I', 28),
 ('don t even', 27),
 ('and I have', 26),
 ('I used to', 26),
 ('but it s', 26),
 ('I feel so', 26),
 ('but I 

In [248]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(corpus)
# print(word_count_vector)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)


# print idf values
df_idf = pd.DataFrame(zip(cv.get_feature_names(),tfidf_transformer.idf_), columns=["word","idf_weights"])
 
# sort ascending
df_idf.sort_values(by='idf_weights',ascending=False)

df_idf.to_csv('test_idf.csv')
df_idf.sort_values(by='idf_weights',ascending=True)

,word,idf_weights
5571,to,1.423536
325,and,1.517534
3614,my,1.597577
5469,the,1.629325
3775,of,1.659341
...,...,...
3865,overcooked,6.855072
3866,overdosed,6.855072
3867,overdosing,6.855072
3869,overlook,6.855072


In [249]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(corpus)


In [250]:
first_document_vector = tfidf_vectorizer_vectors[0]
df = pd.DataFrame(first_document_vector.T.todense(),  columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
2919,0.425494
1534,0.362222
3584,0.296242
3064,0.258363
5945,0.251087
...,...
2065,0.000000
2064,0.000000
2063,0.000000
2062,0.000000


In [262]:
# It looks like below code has no purpose but maybe we can use it later
word_count_vector=cv.fit_transform(lemmatized_wordslist)

In [260]:
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(lemmatized_wordslist)

In [261]:
tfidf_vectorizer_vectors

<43122x5625 sparse matrix of type '<class 'numpy.float64'>'
	with 42748 stored elements in Compressed Sparse Row format>